In [35]:
import pandas as pd
import joblib
import json
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss

In [36]:
# load data
df = pd.read_csv("cblol_training_dataV2.csv")

In [37]:
# champ encoding
champ_cols = [col for col in df.columns if 'champion' in col]
all_champs = pd.concat([df[col].astype(str) for col in champ_cols]).unique()
all_champs = np.append(all_champs, 'Unknown') 

In [38]:
encoder = LabelEncoder()
encoder.fit(all_champs)

LabelEncoder()

In [39]:
for col in champ_cols:
    df[col] = df[col].astype(str).apply(lambda x: x if x in encoder.classes_ else 'Unknown')
    df[col] = encoder.transform(df[col])

In [40]:
joblib.dump(encoder, "champion_label_encoder.pkl")

['champion_label_encoder.pkl']

In [41]:
# freature engineering gaps
roles = ['top', 'jng', 'mid', 'bot', 'sup']
for r in roles:
    df[f'{r}_mastery_gap'] = df[f'Blue_{r}_player_champ_wr'] - df[f'Red_{r}_player_champ_wr']
    df[f'{r}_form_gap'] = df[f'Blue_{r}_player_recent_form'] - df[f'Red_{r}_player_recent_form']

In [42]:
df['team_strength_gap'] = df['Blue_Team_Strength'] - df['Red_Team_Strength']

In [43]:
# select feature and save target
# drop metadata
metadata_cols = ['gameid', 'date', 'Blue_Team', 'Red_Team', 'blue_win_label', 
                 'patch', 'patch_blue', 'patch_red', 'teamname_red', 'teamname_blue', 'result']
X = df.drop(columns=[c for c in metadata_cols if c in df.columns])
X = X.select_dtypes(include=[np.number]) # Keep only numbers

In [44]:
y = df['blue_win_label']

In [45]:
# split and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

In [46]:
# model definition
model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,       
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

In [47]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=-1, num_parallel_tree=None, ...)

In [48]:
# evaluation
preds = model.predict(X_test)
probs = model.predict_proba(X_test)[:, 1] # Probability of Blue Win

In [49]:
acc = accuracy_score(y_test, preds)
loss = log_loss(y_test, probs)

In [50]:
print(f"\nACCURACY: {acc:.2%}")
print(f"Log Loss (Confidence Error): {loss:.4f} (Lower is better)")
print("\n" + classification_report(y_test, preds))


ACCURACY: 59.48%
Log Loss (Confidence Error): 0.7063 (Lower is better)

              precision    recall  f1-score   support

           0       0.50      0.53      0.52        62
           1       0.67      0.64      0.65        91

    accuracy                           0.59       153
   macro avg       0.58      0.58      0.58       153
weighted avg       0.60      0.59      0.60       153



In [51]:
print("💾 Salvando arquivos do modelo...")
# save model
model.get_booster().save_model("cblol_predictor.json")

# Save feature list
import json
feature_list = list(X.columns)
with open("model_features.json", "w") as f:
    json.dump(feature_list, f)

💾 Salvando arquivos do modelo...
